In [1]:
# Copyright (c) Meta Platforms, Inc. All Rights Reserved
import sys
sys.path.insert(0, '/userhome/42/msd21003/TATS')

import os
import argparse
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from tats import Net2NetTransformer, VideoData

In [2]:
pl.seed_everything(1234)

parser = argparse.ArgumentParser()
parser = pl.Trainer.add_argparse_args(parser)
parser = Net2NetTransformer.add_model_specific_args(parser)
parser = VideoData.add_data_specific_args(parser)

args = parser.parse_args(args=["--num_workers", "32", "--val_check_interval", " 0.5", "--progress_bar_refresh_rate", " 500",
                    "--gpus", " 8" ,"--sync_batchnorm" ,"--batch_size", " 3",  "--unconditional",
                    "--vqvae", " ../../ckpt/vqgan_ucf.ckpt", "--data_path", " ../../ucf101", "--dataset", "ucf101", "--default_root_dir", " ../../trainGPT_ckpt",
                    "--vocab_size", " 16384", "--block_size", " 1024", "--n_layer", " 24", "--n_head", " 16", "--n_embd", " 1024",
                    "--resolution", " 128", "--sequence_length", " 16", "--max_steps", " 2000000"])

data = VideoData(args)
# pre-make relevant cached files if necessary
data.train_dataloader()
data.test_dataloader()

args.class_cond_dim = data.n_classes if not args.unconditional and args.cond_stage_key=='label' else None
model = Net2NetTransformer(args, first_stage_key=args.first_stage_key, cond_stage_key=args.cond_stage_key)

callbacks = []
callbacks.append(ModelCheckpoint(every_n_train_steps=10000, save_top_k=-1, filename='{epoch}-{step}-{train/loss:.2f}'))
callbacks.append(ModelCheckpoint(every_n_train_steps=50000, save_top_k=-1, filename='{epoch}-{step}-{train/loss:.2f}'))
callbacks.append(ModelCheckpoint(monitor='val/loss', mode='min', save_top_k=3, filename='best_checkpoint'))

kwargs = dict()
if args.gpus > 1:
    # find_unused_parameters = False to support gradient checkpointing
    kwargs = dict(gpus=args.gpus,
                  # plugins=["deepspeed_stage_2"])
                  plugins=[pl.plugins.DDPPlugin(find_unused_parameters=False)])

# configure learning rate
bs, base_lr = args.batch_size, args.base_lr
ngpu = args.gpus
accumulate_grad_batches = args.accumulate_grad_batches or 1
print(f"accumulate_grad_batches = {accumulate_grad_batches}")
model.learning_rate = accumulate_grad_batches * ngpu * bs * base_lr
print("Setting learning rate to {:.2e} = {} (accumulate_grad_batches) * {} (num_gpus) * {} (batchsize) * {:.2e} (base_lr)".format(
    model.learning_rate, accumulate_grad_batches, ngpu, bs, base_lr))

# load the most recent checkpoint file
base_dir = os.path.join(args.default_root_dir, 'lightning_logs')
if os.path.exists(base_dir):
    log_folder = ckpt_file = ''
    version_id_used = step_used = 0
    for folder in os.listdir(base_dir):
        version_id = int(folder.split('_')[1])
        if version_id > version_id_used:
            version_id_used = version_id
            log_folder = folder
    if len(log_folder) > 0:
        ckpt_folder = os.path.join(base_dir, log_folder, 'checkpoints')
        for fn in os.listdir(ckpt_folder):
            if fn == 'latest_checkpoint.ckpt':
                ckpt_file = 'latest_checkpoint_prev.ckpt'
                os.rename(os.path.join(ckpt_folder, fn), os.path.join(ckpt_folder, ckpt_file))
        if len(ckpt_file) > 0:
            args.resume_from_checkpoint = os.path.join(ckpt_folder, ckpt_file)
            print('will start from the recent ckpt %s'%args.resume_from_checkpoint)

trainer = pl.Trainer.from_argparse_args(args, callbacks=callbacks,
                                        max_steps=args.max_steps, **kwargs)

trainer.fit(model, data)

Global seed set to 1234


data_path:<class 'str'>,sequence_len:<class 'int'>,dataset:<class 'str'>,train:<class 'bool'>,dataset:<class 'type'>


TypeError: __init__() got an unexpected keyword argument 'istrain'